Step 1: Import the necessary libraries


# BERT based distributed training

70p-30np Distribution



## Total Object Models: 9


## Training

1.	Camping
2.	Customer_Order
3.	Ecommerce
4.	Onlinestore
5.	Decider
6.  Library OM
7.  CSOS
8.  Flagship

## Testing

9.	Bank





--------------------------------

## Total Training Data: 13236 (100%)
--------------------------------

### Training set P :  9265 (70% of Training Data)

### Training set NP : 3971 (30% of Training Data)

-----------------------------
## Total Testing Data: 32 (100% of Total Data)
----------------------------

### Testing set P : 8 (25% of Testing Data)

### Testing set NP : 24 (75%% of Testing Data)

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

Load Data

In [2]:
def load_imdb_data(data_file):
    df = pd.read_csv(data_file)
    texts = df['OM_Regular'].tolist()
    labels = [1 if sentiment == "P" else 0 for sentiment in df['OM_Prediction'].tolist()]
    return texts, labels

In [3]:
data_file = "raw_8_om_training_set.csv"
texts, labels = load_imdb_data(data_file)

Step 3: Create a custom dataset class for text classification

In [4]:
class TextClassificationDataset(Dataset):
      def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
      def __len__(self):
        return len(self.texts)
      def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}

Step 4: Build our custom BERT classifier

In [5]:
class BERTClassifier(nn.Module):
  def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

  def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

Step 5: Define the train() function

In [6]:
def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

Step 6: Build our evaluation method


In [7]:
def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    return accuracy_score(actual_labels, predictions), classification_report(actual_labels, predictions)

Step 7: Build our prediction method

In [8]:
def predict_sentiment(text, model, tokenizer, device, max_length=500):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
        return "P" if preds.item() == 1 else "NP"

Step 8: Define our model’s parameters

In [9]:
# Set up parameters
bert_model_name = 'bert-base-uncased'
num_classes = 2
max_length = 500
batch_size = 16
num_epochs = 5
learning_rate = 2e-5

Step 9: Loading and splitting the data.

In [10]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)


Step 10: Initialize tokenizer, dataset, and data loader

In [11]:
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = TextClassificationDataset(val_texts, val_labels, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Step 11: Set up the device and model

In [12]:
#torch.cuda.empty_cache()

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier(bert_model_name, num_classes).to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Step 12: Set up optimizer and learning rate scheduler

In [14]:
# optimizer = AdamW(model.parameters(), lr=learning_rate, no_deprecation_warning=True)
# total_steps = len(train_dataloader) * num_epochs
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

In [15]:
optimizer =  AdamW(model.parameters(), lr=learning_rate, weight_decay=0.02, no_deprecation_warning=True)
total_steps = len(train_dataloader) * num_epochs
num_warmup_steps = total_steps * 0.3  # Adjust to see whether it improves performance

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=total_steps)

Step 13: Training the model

In [16]:
for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        train(model, train_dataloader, optimizer, scheduler, device)
        accuracy, report = evaluate(model, val_dataloader, device)
        print(f"Validation Accuracy: {accuracy:.4f}")
        print(report)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Accuracy: 0.7118
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       769
           1       0.71      1.00      0.83      1899

    accuracy                           0.71      2668
   macro avg       0.36      0.50      0.42      2668
weighted avg       0.51      0.71      0.59      2668

Epoch 2/5


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Accuracy: 0.7118
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       769
           1       0.71      1.00      0.83      1899

    accuracy                           0.71      2668
   macro avg       0.36      0.50      0.42      2668
weighted avg       0.51      0.71      0.59      2668

Epoch 3/5


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Accuracy: 0.7118
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       769
           1       0.71      1.00      0.83      1899

    accuracy                           0.71      2668
   macro avg       0.36      0.50      0.42      2668
weighted avg       0.51      0.71      0.59      2668

Epoch 4/5


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Accuracy: 0.7118
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       769
           1       0.71      1.00      0.83      1899

    accuracy                           0.71      2668
   macro avg       0.36      0.50      0.42      2668
weighted avg       0.51      0.71      0.59      2668

Epoch 5/5
Validation Accuracy: 0.7118
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       769
           1       0.71      1.00      0.83      1899

    accuracy                           0.71      2668
   macro avg       0.36      0.50      0.42      2668
weighted avg       0.51      0.71      0.59      2668



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Saving the trained model

In [17]:
torch.save(model.state_dict(), "bert_classifier_bank.pth")

In [18]:
# # ORM status prediction
# test_text = ""
# sentiment = predict_sentiment(test_text, model, tokenizer)
# print("")
# print(f"status: {sentiment}")

Step 14: Evaluating our model’s performance

In [19]:
import pandas as pd

# Load CSV file
df = pd.read_excel("Bank_authentic_testset.xlsx")  # Update with your file path

# Assuming 'OM_Regular' is the column name
texts = df['OM_Regular']

# Iterate through each text in the column
for test_text in texts:
    sentiment = predict_sentiment(test_text, model, tokenizer, device)
    print("")
    print(f"status: {sentiment}")



status: P

status: P

status: P

status: P

status: P

status: P

status: P

status: P

status: P

status: P

status: P

status: P

status: P

status: P

status: P

status: P

status: P

status: P

status: P

status: P

status: P

status: P

status: P

status: P

status: P

status: P

status: P

status: P

status: P

status: P

status: P

status: P


Calculating Results from separate Testset

In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve, classification_report

In [2]:
dc = pd.read_excel('Bank_authentic_testset.xlsx')

In [3]:
X_test2 = dc['OM_Regular'].values
y_test2 = dc['OM_Prediction'].values

In [4]:
print(X_test2.shape)
print(y_test2.shape)

print("X data type: ", X_test2.dtype)
print("y data type: ", y_test2.dtype)

(32,)
(32,)
X data type:  object
y data type:  int64


In [5]:
print(y_test2)

[1 1 1 1 0 1 1 1 0 0 1 0 1 1 0 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 0]


In [6]:
dd = pd.read_excel('Bank_authentic_testset_pred_6.xlsx')

In [7]:
X_test_pred2 = dd['OM_Regular'].values
y_test_pred2 = dd['OM_Prediction'].values

In [8]:
print (y_test_pred2 )

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [9]:
precision = precision_score(y_test2, y_test_pred2)
print("Testing: Precision = %f" % precision)


recall = recall_score(y_test2, y_test_pred2)
print("Testing: Recall = %f" % recall)


f1 = f1_score(y_test2, y_test_pred2)
print("Testing: F1 Score = %f" % f1)

print("\nConfusion Matrix (Test Data):\n", confusion_matrix(y_test2, y_test_pred2))

Testing: Precision = 0.593750
Testing: Recall = 1.000000
Testing: F1 Score = 0.745098

Confusion Matrix (Test Data):
 [[ 0 13]
 [ 0 19]]


In [10]:
print(classification_report(y_test2,y_test_pred2))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        13
           1       0.59      1.00      0.75        19

    accuracy                           0.59        32
   macro avg       0.30      0.50      0.37        32
weighted avg       0.35      0.59      0.44        32



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
